## Part 2 Instructions

### Have Customers Narrow Their Travel Searches Based on Temperature and Precipitation

In [1]:
# Import the dependencies.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gmaps

# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

# File to load
vacation_weather_to_load = "weather_data/WeatherPy_challenge.csv"

# Import the gmaps API key.
from config import g_key

In [2]:
# Read the WeatherPy_challenge data file and store it in a Pandas DataFrame.
vacation_weather_df = pd.read_csv(vacation_weather_to_load)
vacation_weather_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,0,Raahe,FI,64.68,24.48,33.01,99,90,2.24,light rain and snow,0.0,0.0
1,1,Lebu,CL,-37.62,-73.65,54.23,90,9,3.06,clear sky,0.0,0.0
2,2,Kukuna,SL,9.40,-12.67,78.24,67,4,3.44,clear sky,0.0,0.0
3,3,Ushuaia,AR,-54.80,-68.30,55.40,87,75,36.91,shower rain,0.0,0.0
4,4,Flinders,AU,-34.58,150.86,77.00,60,75,10.29,broken clouds,0.0,0.0


### Find the customer's ideal range of temperatures for travel.

In [3]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# Filter the dataset to find the cities that fit the temperature criteria.
preferred_cities_temp_df = vacation_weather_df.loc[(vacation_weather_df["Max Temp"] <= max_temp) & \
                                       (vacation_weather_df["Max Temp"] >= min_temp)]
preferred_cities_temp_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
2,2,Kukuna,SL,9.40,-12.67,78.24,67,4,3.44,clear sky,0.00,0.0
4,4,Flinders,AU,-34.58,150.86,77.00,60,75,10.29,broken clouds,0.00,0.0
6,6,Busselton,AU,-33.65,115.33,80.01,48,16,10.92,few clouds,0.00,0.0
9,9,Puerto Ayora,EC,-0.74,-90.35,80.60,83,40,11.41,scattered clouds,0.00,0.0
17,17,Hilo,US,19.73,-155.09,82.40,54,1,13.87,clear sky,0.00,0.0
23,23,Rikitea,PF,-23.12,-134.97,80.11,65,2,7.78,light rain,0.19,0.0
27,27,Pinotepa Nacional,MX,16.32,-98.02,83.08,42,0,5.10,clear sky,0.00,0.0
36,36,Mattru,SL,7.62,-11.83,78.49,86,30,1.88,scattered clouds,0.00,0.0
41,41,Souillac,MU,-20.52,57.52,75.99,94,20,2.24,few clouds,0.00,0.0
42,42,Faanui,PF,-16.48,-151.75,83.07,71,34,9.46,light rain,0.31,0.0


### Find out the customer's preferences for rain and snow.

In [5]:
# Ask the customer if they want it to be raining.
raining = input("Do you want it to be raining? (y/n)")
# Ask the customer if they want it to be snowing.
snowing = input("Do you want it to be snowing? (y/n)")

Do you want it to be raining? (y/n)n
Do you want it to be snowing? (y/n)n


In [6]:
# Filter the dataset to find cities with rain.
if raining == "y" and snowing == "y":
    preferred_cities_temp_rain_snow_df = preferred_cities_temp_df.loc[(preferred_cities_temp_df["Rain inches (last 3 hrs)"] > 0) & 
                                                                      (preferred_cities_temp_df["Snow inches (last 3 hrs)"] > 0)]

elif raining == "n" and snowing == "y": 
    preferred_cities_temp_no_rain_snow_df = preferred_cities_temp_df.loc[(preferred_cities_temp_df["Rain inches (last 3 hrs)"] == 0) &
                                                                        (preferred_cities_temp_df["Snow inches (last 3 hrs)"] > 0)]

elif raining == "y" and snowing == "n":
    preferred_cities_temp_rain_no_snow_df = preferred_cities_temp_df.loc[(preferred_cities_temp_df["Rain inches (last 3 hrs)"] > 0) &
                                                                         (preferred_cities_temp_df["Snow inches (last 3 hrs)"] == 0)]
    
else:
    preferred_cities_temp_no_rain_no_snow_df = preferred_cities_temp_df.loc[(preferred_cities_temp_df["Rain inches (last 3 hrs)"] == 0) &
                                                                            (preferred_cities_temp_df["Snow inches (last 3 hrs)"] == 0)]

### Create a marker layer map with a pop-up marker for each city meeting the customer's travel criteria to include hotel name, city, country, and current weather description.

In [7]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [8]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_temp_no_rain_no_snow_df[["City", "Country", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(5)

,City,Country,Current Description,Lat,Lng,Hotel Name
2,Kukuna,SL,clear sky,9.40,-12.67,
4,Flinders,AU,broken clouds,-34.58,150.86,
6,Busselton,AU,few clouds,-33.65,115.33,
9,Puerto Ayora,EC,scattered clouds,-0.74,-90.35,
17,Hilo,US,clear sky,19.73,-155.09,


In [9]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()
hotels

{'html_attributions': [],
 'next_page_token': 'CqQCHgEAAJBhCbBZnsH_7cusZxArsTvau4_opFrxKPGidyfyIg4UaamH4Y3Jn7V04hNg7rD_XLovw1ypjvzNH98aykgUOLqNXil2tq0FG21_sN5vnoK9kJ6fhgNZKlGYUDZj1K-VjH3nCyaoVAwrXEXtcloSL5RsCgvp-aHkeNdWDStksieI-WZqK2Q1jgQKKbrejnClSVOgwSNWSfoX_ehOwcuZbot5HyLkxZk2lUK_MO_m7LjhIh6f3g_M6agYXZYflrQfqe0SV6Ws_1I7Sju9mVvjeJw6g3DOmgjeaU7v9pGLCJHx5vJy2RKC5KcBoyYeLhtIR1poV_rBs0WGG3GdsXMlJ2Qq_cjYhPvVdyDWXGnHd4TlXlWcRu1n00UhztzXrbJm1RIQtjBYXu7MAp8yK6C_5I4GYxoUIlgNkLIRjfCPscPUxy7KWlvwPNg',
 'results': [{'geometry': {'location': {'lat': 48.8581126, 'lng': 2.3529277},
    'viewport': {'northeast': {'lat': 48.8594402302915,
      'lng': 2.354348530291502},
     'southwest': {'lat': 48.8567422697085, 'lng': 2.351650569708498}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
   'id': '7fa47cb61d7345cecc3b802d0e04fcff2b3d5ecb',
   'name': 'Hôtel Duo',
   'opening_hours': {'open_now': True},
   'photos': [{'height': 3840,
     'html_attributions': ['<a href=

In [15]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
   
    # Grab the first hotel from the results and store the name.

    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping.")

        

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [16]:
hotel_df.head(5)

,City,Country,Current Description,Lat,Lng,Hotel Name
2,Kukuna,SL,clear sky,9.40,-12.67,
4,Flinders,AU,broken clouds,-34.58,150.86,Shellharbour Resort & Conference Centre
6,Busselton,AU,few clouds,-33.65,115.33,Observatory Guest House
9,Puerto Ayora,EC,scattered clouds,-0.74,-90.35,Finch Bay Galapagos Hotel
17,Hilo,US,clear sky,19.73,-155.09,Hilo Hawaiian Hotel


In [ ]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

In [19]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [20]:
# Add a map with hotels for the vacation spots and a pop-up marker for each hotel.
locations = hotel_df[["Lat", "Lng"]]
hotel = hotel_df["Hotel Name"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))